In [9]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import osmnx as ox
import warnings
warnings.filterwarnings('ignore')
ox.__version__



'1.9.1'

In [29]:
#get sample
add = "Pohjois-Tapiola, Espoo"
radius = 1500

def get_landuse(add,radius):
    tags = {'natural':True,'landuse':True}
    data = ox.features_from_address(add,dist=radius,tags=tags).reset_index()
    gdf = data.loc[data['geometry'].geom_type.isin(['Polygon', 'MultiPolygon'])]
    gdf['type'] = gdf.apply(lambda row: row['landuse'] if pd.notna(row['landuse']) else row['natural'], axis=1)
    #clip
    loc = ox.geocode(add)
    center_gdf = gpd.GeoDataFrame(geometry=[Point(loc[1],loc[0])], crs="EPSG:4326")
    utm = center_gdf.estimate_utm_crs()
    gdf_clipped = gpd.clip(gdf.to_crs(utm), center_gdf.to_crs(utm).buffer(radius))
    #cols
    columns=['osmid', 'name','type','subtype','geometry']
    selected_columns = [col for col in columns if col in gdf.columns]
    return  gdf_clipped.to_crs(4326)[selected_columns]

sample = get_landuse(add,radius)
sample

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/usr/local/lib/python3.10/dist-packages/pyproj/transformer.py:820: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return self._transformer._transform_point(


,osmid,name,type,geometry
2210,16269929,NaN,residential,"POLYGON ((24.79929 60.17326, 24.79975 60.17316..."
2209,16269928,NaN,construction,"POLYGON ((24.79847 60.17274, 24.79889 60.17303..."
1718,854896413,NaN,wood,"POLYGON ((24.79844 60.17272, 24.79784 60.17267..."
1716,854639359,NaN,grass,"POLYGON ((24.80074 60.17311, 24.80081 60.17312..."
2141,1152745401,NaN,construction,"POLYGON ((24.80764 60.17452, 24.80809 60.17459..."
...,...,...,...,...
1490,766842681,NaN,grass,"POLYGON ((24.78700 60.19889, 24.78693 60.19886..."
1915,1025326367,NaN,scrub,"POLYGON ((24.78788 60.19879, 24.78777 60.19879..."
1916,1025326368,NaN,grass,"POLYGON ((24.78757 60.19880, 24.78757 60.19908..."
1920,1025326372,NaN,meadow,"POLYGON ((24.78818 60.19879, 24.78815 60.19877..."


In [30]:
sample.fillna('nan').explore('type',tiles="CartoDB positron")